# **Sistema de Recomendação de Música**

Um Sistema de Recomendação Musical é uma aplicação de Data Science que visa auxiliar os usuários na descoberta de conteúdos musicais novos e relevantes com base em suas preferências e comportamento auditivo.  As recomendações musicais personalizadas tornaram-se uma ferramenta essencial no cenário da música digital, permitindo que plataformas de streaming de música como Spotify e Apple Music ofereçam experiências personalizadas e envolventes aos seus utilizadores.

## Token de Acesso

Para começar a construir um sistema de recomendação musical, primeiro precisamos ter um token de acesso.  O token de acesso serve como uma credencial de autorização temporária, permitindo que o código faça solicitações autenticadas à API do Spotify em nome do aplicativo.

**Importando as Bibliotecas**

In [149]:
# !pip install spotipy
# !pip install scikit-learn

In [150]:
import requests
import base64

In [151]:
# Substitua pelo seu próprio Client ID e Client Secret
CLIENT_ID = 'dc24d2ff39cd4bc4aaf087c89d8f6293'
CLIENT_SECRET = 'aa13ae7c60b2465e998bd6b5b62d9328'

In [152]:
# Base64 codifica o ID do cliente e o segredo do cliente
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [153]:
# Solicite o token de acesso
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Token de acesso obtido com sucesso.")
else:
    print("Erro ao obter token de acesso.")
    exit()

Token de acesso obtido com sucesso.


In [154]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [155]:
def get_trending_playlist_data(playlist_id, access_token):
    # Configure o Spotipy com o token de acesso
    sp = spotipy.Spotify(auth=access_token)

    # Obtenha as faixas da playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name))')

    # Extraia informações relevantes e armazene em uma lista de dicionários
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Obtenha recursos de áudio para a faixa
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None
    
        # Obtenha a data de lançamento do álbum
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Obtenha popularidade da faixa
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Adicione informações adicionais da trilha aos dados da trilha
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }

        music_data.append(track_data)

    # Crie um DataFrame do pandas a partir da lista de dicionários
    df = pd.DataFrame(music_data)

    return df

In [156]:
playlist_id = '4QeniwUa5dxB6Alf3I8IN3'

# Chame a função para obter os dados musicais da lista de reprodução e armazená-los em um DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Exibir o DataFrame
print(music_df)

                                        Track Name  \
0                  Sweet Dreams (Are Made of This)   
1                                  Miracle of Love   
2   There Must Be an Angel (Playing With My Heart)   
3                              When Tomorrow Comes   
4                        Here Comes the Rain Again   
..                                             ...   
95                                 Lips Like Sugar   
96                                   The Passenger   
97                 Cities In Dust - Single Version   
98                                Kiss Them For Me   
99                                867-5309 / Jenny   

                                   Artists  \
0   Eurythmics, Annie Lennox, Dave Stewart   
1   Eurythmics, Annie Lennox, Dave Stewart   
2   Eurythmics, Annie Lennox, Dave Stewart   
3   Eurythmics, Annie Lennox, Dave Stewart   
4   Eurythmics, Annie Lennox, Dave Stewart   
..                                     ...   
95                     Echo &

In [157]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [158]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [159]:
# Identificando datas com ano
datas_ano = music_df['Release Date'].str.len() == 4

# Convertendo datas com ano para datas completas
music_df.loc[datas_ano, 'Release Date'] = pd.to_datetime(music_df['Release Date'][datas_ano],
                                                         format='%Y', errors='coerce')

# Retirando o sufixo ' 00:00:00' da datas completadas
music_df['Release Date'] = music_df['Release Date'].astype(str)
music_df['Release Date'] = music_df['Release Date'].str.removesuffix(' 00:00:00')

In [160]:
data = music_df

In [161]:
# Função para calcular pontuações de popularidade ponderadas com base na data de lançamento
def calculate_weighted_popularity(release_date):
    # Converter a data de lançamento em objeto datetime
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calcule o intervalo de tempo entre a data de lançamento e a data de hoje
    time_span = datetime.now() - release_date

    # Calcule a pontuação de popularidade ponderada com base no intervalo de tempo (por exemplo, lançamentos mais recentes têm peso maior)
    weight = 1 / (time_span.days + 1)
    return weight

In [162]:
# Normalize os recursos musicais usando a escala Min-Max
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [163]:
# Função para obter recomendações baseadas em conteúdo com base em recursos musicais
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' não encontrado no conjunto de dados. Insira um nome de música válido.")
        return

    # Obtenha o índice da música de entrada no DataFrame de música
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calcule as pontuações de similaridade com base nos recursos musicais (semelhança de cosseno)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Obtenha os índices das músicas mais semelhantes
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Obtenha os nomes das músicas mais semelhantes com base na filtragem baseada em conteúdo
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [164]:
# Função para obter recomendações híbridas com base na popularidade ponderada
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' não encontrado no conjunto de dados. Insira um nome de música válido.")
        return

    # Obtenha recomendações baseadas em conteúdo
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Obtenha a pontuação de popularidade da música de entrada
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calcule a pontuação de popularidade ponderada
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine recomendações baseadas em conteúdo e baseadas em popularidade com base na popularidade ponderada
    hybrid_recommendations = content_based_rec
    dict_recommendations = {
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }
    new_hybrid_recommendations = pd.DataFrame.from_dict(dict_recommendations, orient='index')
    new_hybrid_recommendations = new_hybrid_recommendations.T
    hybrid_recommendations = pd.concat([hybrid_recommendations, new_hybrid_recommendations], ignore_index=True)

    # Classifique as recomendações híbridas com base na pontuação de popularidade ponderada
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remova a música de entrada das recomendações
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [166]:
input_song_name = "Rhythm Of The Night"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Músicas híbridas recomendadas para '{input_song_name}':")
print()
print(recommendations)

Músicas híbridas recomendadas para 'Rhythm Of The Night':

                          Track Name                  Artists  \
1                           Maneater  Daryl Hall & John Oates   
3                         Down Under              Men At Work   
0                  Don't You Want Me         The Human League   
4  Maria Magdalena - Remastered 1992                   Sandra   
2                       Walking Away      Information Society   

            Album Name Release Date Popularity  
1                  H2O   1982-10-04         78  
3    Business As Usual   1981-11-09         75  
0                Dare!   1981-01-01         71  
4     18 Greatest Hits   1992-01-01         54  
2  Information Society   1988-06-21         40  
